수치 미분은 단순하고 구현이 쉬우나 계산 시간이 오래 걸림.

오차역전파법(backpropagation)

5.1
계산 그래프 : 계산 과정을 그래프로 나타낸 것
- 구성 : 노드, 엣지
- 순전파 <-> 역전파
    - 역전파는 미분 계산 시 중요한 역할
- 특징 : 국소적 계산을 전파해 최종 결과를 얻음

- 장점
    1. 중간 계산 결괏값을 모두 저장할 수 있음
    2. ★순전파와 역전파를 활용해 각 변수의 미분을 효율적으로 계산할 수 있음

5.2 연쇄법칙
- 역전파의 계산 절차 : 신호 E에 노드의 국소적 미분을 곱한 후 다음 노드로 전달
- 연쇄법칙 : 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

5.3 역전파
- 덧셈 노드 : 입력 값을 그대로 흘려보냄
- 곱셈 노드 : 상류의 값에 순전파 때의 입력 신호들의 서로 바꾼 값을 곱해 하류로 보냄
    - 순방향 입력 신호의 값 필요 -> 저장해둬야 함

계층 : 신경망의 기능 단위
- Sigmoid(시그모이드 함수), Affine(행렬 내적)
- 클래스로 구현
    - 메소드 : forward()와 backward()
    - 곱셈; MulLayer, 덧셈; AddLayer

In [1]:
# 곱셈 계층
class MulLayer:
    def __init__(self): # 초기화
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y # 입력 x와 y를 서로 바꿔 곱함
        dy = dout * self.x
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

# 계층
mul_apple_layer = MulLayer() # 첫 번째 곱겜; 100원 사과 2개
mul_tax_layer = MulLayer() # 두 번째 곱셈; 소비세 10%

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice) # 총액에서 소비세, 사과 가격
dapple, dapple_num = mul_apple_layer.backward(dapple_price) 

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [4]:
# 덧셈 계층
class AddLayer:
    def __init__(self): # 초기화 필요 없음, 저장할 값이 없으므로
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer() # 사과 가격, 오렌지 가격 더하기
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650
